<a href="https://colab.research.google.com/github/rafaelnduarte/Spotify_Mood_Playlist/blob/master/Retrieving_Data_from_Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# installing spotipy
pip install spotipy -q

In [0]:
# importing the necessary packages
import spotipy 
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
import spotipy.util as util
import sys

In [0]:
# setting up authorization
cid ="867a4d8d1a004f8fa5ebb0e961d34660" 
secret = "9467590b680f492cafe47a755a939504"

username = '22fmcdjlhywkas2zn6gk3y4iy'
scope = 'user-library-read'
authorization_url = 'https://accounts.spotify.com/authorize'
token_url = 'https://accounts.spotify.com/api/token'
client_id_rnd ='867a4d8d1a004f8fa5ebb0e961d34660'
client_secret_rnd ='9467590b680f492cafe47a755a939504'
redirect_uri_rnd ='https://localhost.com/callback/'

token = util.prompt_for_user_token(username,scope,client_id='867a4d8d1a004f8fa5ebb0e961d34660',client_secret='9467590b680f492cafe47a755a939504',redirect_uri='https://localhost.com/callback/')
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [67]:
# token
auth = SpotifyClientCredentials(
    client_id=client_id_rnd,
    client_secret=client_secret_rnd
)

token = auth.get_access_token()
spotify = spotipy.Spotify(auth=token)
print(token)
print(spotify)
token = 'BQAuGvoMD37Cy_6jZnXuDlY-ucZ5b8xc5ZuikDpAxmw8v8UaHxid2IavuQW1fMoYynZr4jp-0nBncgYhT9D9HVPRCy1TjuoXZotX8K_u0sYZB2bbaZ-WfvCVOzok_Vw3NADlWQZmSyQURkrfd4woW58r6ua7rwITpKkjnS9RIjvPaMW_ug23Oz1EXMbMVGbjFx8'

BQBon5Cg6dpAj8nE2gm-SyOT4wNr00cnsC20JoFdTiyCrgusMH_0-yCrG7fbugFRKfJlcf1Un_ajc574bpA


In [68]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,10000,50):
    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 29.380561838001086


In [69]:
# checking the results
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 10000


In [70]:
# putting the data in a dataframe
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(10000, 4)


,artist_name,track_name,track_id,popularity
0,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,99
1,Travis Scott,HIGHEST IN THE ROOM,3eekarcy7kvN4yt5ZFzltW,98
2,Juice WRLD,Bandit (with YoungBoy Never Broke Again),6Gg1gjgKi2AK4e0qzsR7sd,93
3,Arizona Zervas,ROXANNE,696DnlkuDOXcMAnKlTgXXK,97
4,Selena Gomez,Lose You To Love Me,1HfMVBKM75vxSfsQ5VefZ5,98


In [71]:
# info on the dataset
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
artist_name    10000 non-null object
track_name     10000 non-null object
track_id       10000 non-null object
popularity     10000 non-null int64
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


In [72]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

528

In [0]:
# dropping duplicates
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [74]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [75]:
# dropping duplicates
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [76]:
# checking the shape of the dataframe
df_tracks.shape

(9349, 4)

In [77]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 29
Time to run this code (in seconds): 6.935885976999998


In [78]:
# checking the results
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 9320


In [79]:
# putting the data in a dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (9320, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.695,0.762,0,-3.497,1,0.0395,0.1920,0.002440,0.0863,0.5530,120.042,audio_features,21jGcNKet2qwijlDFuPiPb,spotify:track:21jGcNKet2qwijlDFuPiPb,https://api.spotify.com/v1/tracks/21jGcNKet2qw...,https://api.spotify.com/v1/audio-analysis/21jG...,215280,4
1,0.598,0.427,7,-8.764,0,0.0317,0.0546,0.000006,0.2100,0.0605,76.469,audio_features,3eekarcy7kvN4yt5ZFzltW,spotify:track:3eekarcy7kvN4yt5ZFzltW,https://api.spotify.com/v1/tracks/3eekarcy7kvN...,https://api.spotify.com/v1/audio-analysis/3eek...,175721,4
2,0.474,0.631,5,-5.884,0,0.3430,0.0687,0.000000,0.1320,0.4250,180.051,audio_features,6Gg1gjgKi2AK4e0qzsR7sd,spotify:track:6Gg1gjgKi2AK4e0qzsR7sd,https://api.spotify.com/v1/tracks/6Gg1gjgKi2AK...,https://api.spotify.com/v1/audio-analysis/6Gg1...,189323,4
3,0.621,0.601,6,-5.616,0,0.1480,0.0522,0.000000,0.4600,0.4570,116.735,audio_features,696DnlkuDOXcMAnKlTgXXK,spotify:track:696DnlkuDOXcMAnKlTgXXK,https://api.spotify.com/v1/tracks/696DnlkuDOXc...,https://api.spotify.com/v1/audio-analysis/696D...,163636,5
4,0.505,0.340,4,-9.005,1,0.0438,0.5760,0.000000,0.2100,0.0916,101.993,audio_features,1HfMVBKM75vxSfsQ5VefZ5,spotify:track:1HfMVBKM75vxSfsQ5VefZ5,https://api.spotify.com/v1/tracks/1HfMVBKM75vx...,https://api.spotify.com/v1/audio-analysis/1HfM...,206459,4


In [80]:
# info on the dataframe
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9320 entries, 0 to 9319
Data columns (total 18 columns):
danceability        9320 non-null float64
energy              9320 non-null float64
key                 9320 non-null int64
loudness            9320 non-null float64
mode                9320 non-null int64
speechiness         9320 non-null float64
acousticness        9320 non-null float64
instrumentalness    9320 non-null float64
liveness            9320 non-null float64
valence             9320 non-null float64
tempo               9320 non-null float64
type                9320 non-null object
id                  9320 non-null object
uri                 9320 non-null object
track_href          9320 non-null object
analysis_url        9320 non-null object
duration_ms         9320 non-null int64
time_signature      9320 non-null int64
dtypes: float64(9), int64(4), object(5)
memory usage: 1.3+ MB


In [81]:
# dropping unnecessary columns
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(9320, 14)

In [82]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (9320, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,99,0.695,0.762,0,-3.497,1,0.0395,0.1920,0.002440,0.0863,0.5530,120.042,215280,4
1,Travis Scott,HIGHEST IN THE ROOM,3eekarcy7kvN4yt5ZFzltW,98,0.598,0.427,7,-8.764,0,0.0317,0.0546,0.000006,0.2100,0.0605,76.469,175721,4
2,Juice WRLD,Bandit (with YoungBoy Never Broke Again),6Gg1gjgKi2AK4e0qzsR7sd,93,0.474,0.631,5,-5.884,0,0.3430,0.0687,0.000000,0.1320,0.4250,180.051,189323,4
3,Arizona Zervas,ROXANNE,696DnlkuDOXcMAnKlTgXXK,97,0.621,0.601,6,-5.616,0,0.1480,0.0522,0.000000,0.4600,0.4570,116.735,163636,5
4,Selena Gomez,Lose You To Love Me,1HfMVBKM75vxSfsQ5VefZ5,98,0.505,0.340,4,-9.005,1,0.0438,0.5760,0.000000,0.2100,0.0916,101.993,206459,4


In [83]:
# info on the merge dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9320 entries, 0 to 9319
Data columns (total 17 columns):
artist_name         9320 non-null object
track_name          9320 non-null object
track_id            9320 non-null object
popularity          9320 non-null int64
danceability        9320 non-null float64
energy              9320 non-null float64
key                 9320 non-null int64
loudness            9320 non-null float64
mode                9320 non-null int64
speechiness         9320 non-null float64
acousticness        9320 non-null float64
instrumentalness    9320 non-null float64
liveness            9320 non-null float64
valence             9320 non-null float64
tempo               9320 non-null float64
duration_ms         9320 non-null int64
time_signature      9320 non-null int64
dtypes: float64(9), int64(5), object(3)
memory usage: 1.3+ MB


In [84]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [0]:
# creating a csv file
df.to_csv('spotify_data.csv')

In [0]:
from google.colab import files


In [0]:
# downloading the csv file
files.download("spotify_data.csv")